!pip install yfinance

!pip install pandas_datareader

In [1]:
# Manipulação de Dados
import numpy as np
import pandas as pd
from math import nan
from datetime import datetime
# Obtenção das cotações
import pandas_datareader.data as web
import yfinance as yf
import os

In [2]:
def buscar_cotacoes(arquivo_txt, dias_cotacoes, country):

    tickers_df = pd.read_csv(arquivo_txt, header=None)
    tickers = []
    if country == "US":
        casas_arred = 4
        for ticker in tickers_df[0]:
            tickers.append(ticker)
    elif country == "BR":
        casas_arred = 0
        for ticker in tickers_df[0]:
            tickers.append(ticker + ".SA")

    yf.pdr_override()
    cotations = web.get_data_yahoo(tickers, period=str(dias_cotacoes) + 'd' , threads=1)['Adj Close']
    
    return cotations, tickers, casas_arred

In [3]:
def calcular_variacoes(cotations, tickers):
    
    n_lins = cotations.shape[0]
    n_cols = cotations.shape[1]
    
    cotations_var = np.zeros(shape=(n_lins - 1, n_cols), dtype=float)
    cotations_var = pd.DataFrame(data=cotations_var, columns=tickers)
    for ticker in tickers:
        for j in range(1, n_lins):
            cotations_var[ticker][j - 1] = (cotations[ticker][j] - cotations[ticker][j - 1]) / cotations[ticker][j - 1]
    
    return cotations_var

In [4]:
def gerar_cromos(num_cols, fixar_seed=False, seed=42, num_cromossomos=6):
    if fixar_seed is True:
        np.random.seed(seed)
    
    cromossomos = np.random.rand(num_cromossomos, num_cols)

    for i in range(cromossomos.shape[0]):
        cromossomos[i] = cromossomos[i] / cromossomos[i].sum()
    
    return cromossomos

In [5]:
def calc_retornos_riscos_carteira(cromos, medias, mat_cov):
    retornos = cromos.dot(medias)
    
    num_cromossomos = len(cromos)
    
    riscos = np.zeros(shape=(num_cromossomos))
    for i in range(num_cromossomos):
        riscos[i] = cromos[i].dot(mat_cov).dot(cromos[i].T)
    
    return retornos, riscos

In [6]:
def roda_acaso(fitness_acumulado):
    indices_sorteados = [np.inf]
    while len(indices_sorteados) < 3:
        alfa = np.random.rand()
        for i in range(fitness_acumulado.shape[0]):
            if alfa <= fitness_acumulado[i] and indices_sorteados[-1] != i:
                indices_sorteados.append(i)
                break
    
    return indices_sorteados[1:]

In [7]:
def mutacao_um(cromo, posicoes):
    cromossomo_final = cromo.copy()
    valor_um = cromossomo_final[posicoes[0]]
    valor_dois = cromossomo_final[posicoes[1]]
    cromossomo_final[posicoes[0]] = valor_dois
    cromossomo_final[posicoes[1]] = valor_um
    return cromossomo_final

In [8]:
def mutacao_dois(cromo, posicoes):
    cromo1 = cromo.copy()
    cromo2 = cromo.copy()
    valor_um = cromo[posicoes[0]]
    valor_dois = cromo[posicoes[1]]
    cromo1[posicoes[0]] = 0
    cromo1[posicoes[1]] = valor_um + valor_dois
    cromo2[posicoes[0]] = valor_um + valor_dois
    cromo2[posicoes[1]] = 0
    return cromo1, cromo2

In [9]:
def corrigir_fitnesses(fitnesses):
    fitnesses_corrigidos = []
    for fitness in fitnesses:
        if fitness < -1:
            fitnesses_corrigidos.append(1 / np.abs(fitness))
        elif fitness < 0:
            fitnesses_corrigidos.append(np.abs(fitness))
        else:
            fitnesses_corrigidos.append(fitness)
    
    return np.array(fitnesses_corrigidos)

In [10]:
def substituir_geracoes(fit_pais, fit_filhos, cromos_pais, cromos_filhos):
    args_min_pais = fit_pais.argsort()[:2]
    args_max_filhos = fit_filhos.argsort()[-2:]
    cromos_finais = cromos_pais.copy()
    
    if fit_filhos[args_max_filhos[-1]] > fit_pais[args_min_pais[0]]:
        cromos_finais[args_min_pais[0]] = cromos_filhos[args_max_filhos[-1]]
    elif fit_filhos[args_max_filhos[-1]] > fit_pais[args_min_pais[1]]:
        cromos_finais[args_min_pais[1]] = cromos_filhos[args_max_filhos[-1]]
    else:
        return cromos_finais
    
    if fit_filhos[args_max_filhos[-2]] > fit_pais[args_min_pais[1]]:
        cromos_finais[args_min_pais[1]] = cromos_filhos[args_max_filhos[-2]]

    return cromos_finais

In [11]:
def exportar_df(valor_inv, arr, names_indexes, path_name, perc_corte, casas_arred):
    df = pd.DataFrame(data=(np.round(arr, 2)).T, index=names_indexes, columns=["%"])
    
    for ticker in names_indexes:
        if df["%"][ticker] < 0.05:
            df.drop(labels=[ticker], inplace=True)
    
    cotations = []
    for ticker in df.index:
        cotation = np.round(web.get_data_yahoo(ticker, period='1d' , threads=1)['Adj Close'][0], 2)
        cotations.append(cotation)
    
    df["precos"] = cotations
    
    df["qtd_comprar"] = np.round((df["%"] * valor_inv) / df["precos"], casas_arred)
    
    df["valor_total"] = df["qtd_comprar"] * df["precos"]
    
    df.to_excel(path_name)

In [12]:
def moneta_ag(arq_txt, dp_final, valor_investimento, percentual_corte, country, exportar_cotacoes):
    
    dias_cots = 34
    cotacoes, tickers, casas_arred = buscar_cotacoes(arquivo_txt=arq_txt, dias_cotacoes=dias_cots, country=country)
    
    if exportar_cotacoes is True:
        cotacoes.to_excel(os.path.join("cotacoes", "cotacoes.xlsx"))
    
    num_cotacoes = cotacoes.shape[0]
    num_genes = cotacoes.shape[1]
    
    cotations_var = calcular_variacoes(cotations=cotacoes, tickers=tickers)
    
    mat_cov = np.cov(cotations_var.T)
    
    medias = np.average(cotations_var, axis=0)
    
    cromossomos = gerar_cromos(num_cols=num_genes, fixar_seed=True)
    
    retornos_carteiras, riscos_carteiras = calc_retornos_riscos_carteira(cromos=cromossomos, medias=medias, mat_cov=mat_cov)
    
    fitness_carteiras = retornos_carteiras / riscos_carteiras
    
    desvio_padrao_carteiras = np.std(fitness_carteiras)

    num_filhos = 8
    cromossomos_filhos = np.zeros(shape=(num_filhos, num_genes), dtype=float)
    desvio_padrao_carteiras = np.inf
    
    iteracoes = 0
    while desvio_padrao_carteiras > dp_final:
        fitnesses_corrigidos = corrigir_fitnesses(fitnesses=fitness_carteiras)
        fitness_acum = np.cumsum(fitnesses_corrigidos) / fitnesses_corrigidos.sum()
        cromo_sorteados = roda_acaso(fitness_acumulado=fitness_acum)
        beta = np.random.rand()
        cromossomos_filhos[0] = cromossomos[cromo_sorteados[0]] * beta + cromossomos[cromo_sorteados[1]] * (1 - beta)
        cromossomos_filhos[1] = cromossomos[cromo_sorteados[0]] * (1 - beta) + cromossomos[cromo_sorteados[1]] * beta

        while True:
            genes_mutacao_um = np.random.choice(a=range(num_genes), size=2)
            genes_mutacao_dois = np.random.choice(a=range(num_genes), size=2)
            if genes_mutacao_um[0] != genes_mutacao_um[1] and genes_mutacao_dois[0] != genes_mutacao_dois[1]:
                cromossomos_filhos[2] = mutacao_um(cromo=cromossomos_filhos[0], posicoes=genes_mutacao_um)
                cromossomos_filhos[3] = mutacao_um(cromo=cromossomos_filhos[1], posicoes=genes_mutacao_dois)
                break

        while True:
            genes_mutacao_um = np.random.choice(a=range(num_genes), size=2)
            genes_mutacao_dois = np.random.choice(a=range(num_genes), size=2)
            if genes_mutacao_um[0] != genes_mutacao_um[1] and genes_mutacao_dois[0] != genes_mutacao_dois[1]:
                cromossomos_filhos[4], cromossomos_filhos[5] = mutacao_dois(cromo=cromossomos_filhos[0], 
                                                                            posicoes=genes_mutacao_um)
                
                cromossomos_filhos[6], cromossomos_filhos[7] = mutacao_dois(cromo=cromossomos_filhos[1], 
                                                                            posicoes=genes_mutacao_dois)
                break

        retornos_filhos, riscos_filhos = calc_retornos_riscos_carteira(cromos=cromossomos_filhos, 
                                                                       medias=medias, mat_cov=mat_cov)

        fitness_filhos = retornos_filhos / riscos_filhos

        cromossomos = substituir_geracoes(fit_pais=fitness_carteiras, fit_filhos=fitness_filhos, 
                                          cromos_pais=cromossomos, cromos_filhos=cromossomos_filhos)

        retornos_carteiras, riscos_carteiras = calc_retornos_riscos_carteira(cromos=cromossomos, 
                                                                             medias=medias, mat_cov=mat_cov)

        fitness_carteiras = retornos_carteiras / riscos_carteiras

        desvio_padrao_carteiras = np.std(fitness_carteiras)
        
        iteracoes += 1


    if (np.round(cromossomos.sum(axis=1), 0) == 1).all():
        
        for i in range(100):
            name_file = os.path.join("resultados", f"resultado_{dias_cots}d_{country}_{i}.xlsx")
            if not os.path.exists(name_file):
                path_export = name_file
                break
        
        exportar_df(valor_inv=valor_investimento, arr=cromossomos[0], names_indexes=tickers, 
                    path_name=path_export, perc_corte=percentual_corte, casas_arred=casas_arred)
        print(f"[INFO] O resultado foi obtido com {iteracoes} iteracoes.")
        print(f"[INFO] O resultado final foi exportado com sucesso para: {path_export}")
        print(f"[INFO] O fitness obtido foi de: {round(np.average(fitness_carteiras), 2)}")
        print(f"[INFO] O retorno esperado é de: {round(np.average(retornos_carteiras), 5) * 100} %")
        print(f"[INFO] O risco esperado é de: {round(np.average(riscos_carteiras), 5) * 100} %")
    else:
        print(f"[INFO] A soma dos percentuais não resulta 100% para todos os cromossomos\n {cromossomos.sum(axis=1)}")

In [15]:
moneta_ag(arq_txt=os.path.join("tickers", "TICKERS_BR.txt"), dp_final=0.01, valor_investimento=3000, 
          percentual_corte=0.05, country="BR", exportar_cotacoes=True)

[*********************100%***********************]  83 of 83 completed


KeyboardInterrupt: 